In [1]:

import pandas as pd 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
data_2 = pd.read_csv(r'Heart_Disease_Prediction.csv')
print(data_2)

data_2=data_2.fillna(0) # fill NAs  with 0 

X = data_2.drop(['Heart Disease'], axis=1)
y=Heart_Disease=data_2["Heart Disease"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

     Age  Sex  Chest pain type   BP  Cholesterol  FBS over 120  EKG results  \
0     70    1                4  130          322             0            2   
1     67    0                3  115          564             0            2   
2     57    1                2  124          261             0            0   
3     64    1                4  128          263             0            0   
4     74    0                2  120          269             0            2   
..   ...  ...              ...  ...          ...           ...          ...   
265   52    1                3  172          199             1            0   
266   44    1                2  120          263             0            0   
267   56    0                2  140          294             0            2   
268   57    1                4  140          192             0            0   
269   67    1                4  160          286             0            2   

     Max HR  Exercise angina  ST depression  Slope 

In [12]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
# Select an algorithm
algorithm = KNeighborsClassifier()
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)
# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]
# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
knn =grid.fit(X_train, y_train)

# Get the results
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_) 
# Create 3 folds

knn_predictions = knn.predict(X_test) 
cm = confusion_matrix(y_test, knn_predictions)
print(cm)
classification_report(y_test, knn_predictions)
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print(accuracy)


0.788342525
[[40  9]
 [ 8 33]]
              precision    recall  f1-score   support

     Absence       0.83      0.82      0.82        49
    Presence       0.79      0.80      0.80        41

    accuracy                           0.81        90
   macro avg       0.81      0.81      0.81        90
weighted avg       0.81      0.81      0.81        90



In [10]:
#Naïve Bayes
from sklearn.naive_bayes import GaussianNB
gnb_clf=GaussianNB()
parameters = {
    'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]
}
clf = GridSearchCV(gnb_clf, parameters, cv=5)
gnb=clf.fit(X_train, y_train)


gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(cm)
creport=classification_report(y_test, gnb_predictions)
print(creport)

0.8
[[36 13]
 [ 5 36]]
              precision    recall  f1-score   support

     Absence       0.88      0.73      0.80        49
    Presence       0.73      0.88      0.80        41

    accuracy                           0.80        90
   macro avg       0.81      0.81      0.80        90
weighted avg       0.81      0.80      0.80        90



In [11]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
model = LogisticRegression(solver='liblinear')

c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=1, cv=cv)
grid_result = grid_search.fit(X_train, y_train)


logreg=model.fit(X_train, y_train)
log_predictions = grid_result.predict(X_test)

# accuracy on X_test
accuracy = grid_result.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, log_predictions)
print(cm)
clas_report=classification_report(y_test, log_predictions)
print(clas_report)

0.8111111111111111
[[40  9]
 [ 8 33]]
              precision    recall  f1-score   support

     Absence       0.83      0.82      0.82        49
    Presence       0.79      0.80      0.80        41

    accuracy                           0.81        90
   macro avg       0.81      0.81      0.81        90
weighted avg       0.81      0.81      0.81        90



In [21]:
import pickle
with open('model_heart2','wb') as file:
    pickle.dump(grid_result,file)

In [ ]:
with open('model_heart','rb') as file:
    mp = pickle.load(file)

print(X_test)    
pre=mp.predict([4,90,88,47,54,37.5,0.365,29])
print(pre)

In [13]:
#Decision Tree algorithms
from sklearn.tree import DecisionTreeClassifier
model= DecisionTreeClassifier()
parameters={
            "max_depth" : [7,9,11,12,15],
           "min_samples_leaf":[50,10,100],
           "min_weight_fraction_leaf":[0.0001,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_leaf_nodes":[None,1,20,30,40,50,60,70,80,90] }

tuning_model=GridSearchCV(estimator=model,param_grid=parameters,cv=3,verbose=3)

dtree_model =tuning_model.fit(X_train, y_train)

#dtree_model = DecisionTreeClassifier(max_depth = 15).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
accuracy = dtree_model.score(X_test, y_test)
print(accuracy)  
# creating a confusion matrix
cm = confusion_matrix(y_test, dtree_predictions)
print(cm)

cls_report=classification_report(y_test, dtree_predictions)
print(cls_report)

0.83255222
[[40  9]
 [ 8 33]]
              precision    recall  f1-score   support

     Absence       0.83      0.82      0.82        49
    Presence       0.79      0.80      0.80        41

    accuracy                           0.81        90
   macro avg       0.81      0.81      0.81        90
weighted avg       0.81      0.81      0.81        90



In [1]:
import pickle
import numpy as np 
import pandas as pd
with open('model_heart2','rb') as file:
    mp = pickle.load(file)

In [2]:
X_test_new = [70.0,1.0,4,130.3,322,0,2,109,0,2.4,2,3,3]
print(X_test_new)
datanew=pd.DataFrame(X_test_new).transpose()
pre=mp.predict(datanew)
print(pre)

[70.0, 1.0, 4, 130.3, 322, 0, 2, 109, 0, 2.4, 2, 3, 3]
['Presence']


/home/dilum/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
